# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from sklearn import linear_model
%matplotlib inline

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """Draws a line over a given image."""
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

### Custom Helpers

In [ ]:
def convert_to_hsv(img):
    """Converts given RGB img to HSV format"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

def convert_to_rgb(img):
    """Converts given HSV img to RGB format"""
    return cv2.cvtColor(img, cv2.COLOR_HSV2BGR)

def calculate_hue_saturation_histogram(hsv_img):
    """Calculates the histogram for the hue and saturation channels in an HSV image"""
    return cv2.calcHist([hsv_img], [0, 1], None, [180, 256], [0, 180, 0, 256])

def show_image(img, cmap=None):
    if cmap is not None:
        plt.imshow(img, cmap=cmap)
    else:
        plt.imshow(img)
    plt.show()

def group_lines_in_left_right(img, lines, middle_x_value):
    """
    Separate lines into 2 provided numpy arrays, left and right.
    This ensures we can pass the correct set of lines to RANSAC so it can extrapolate using the correct data.
    """
    left_points = []
    right_points = []
    other_points = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            delta_x = x2 - x1
            delta_y = y2 - y1

            # Vertical line
            if delta_x == 0:
                other_points += [[x1, y1], [x2, y2]]
            else:
                slope = delta_y / delta_x

                # Horizontal line
                if slope == 0:
                    other_points += [[x1, y1], [x2, y2]]

                # Sloping lines that can be categorized by their slope
                # Note: since the coordinate system anchors on the top left, the positive slope goes to the bottom right
                elif slope > 0:
                    right_points += [[x1, y1], [x2, y2]]
                else:
                    left_points += [[x1, y1], [x2, y2]]

    left_points = np.array(left_points)
    right_points = np.array(right_points)

    # Categorize each pair of points defining horizontal/vertical lines as to the left or right of the middle
    for index in range(0, len(other_points) - 1, 2):
        point1 = other_points[index]
        point2 = other_points[index + 1]
        if point1[0] <= middle_x_value:
            left_points = np.append(left_points, [point1, point2], axis=0)
        else:
            right_points = np.append(right_points, [point1, point2], axis=0)

    return left_points, right_points

def create_ransac():
    return linear_model.RANSACRegressor(
        estimator=linear_model.LinearRegression(),
        min_samples=2,
        max_trials=500,
        loss='absolute_error'
    )

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

### Hue Range for Colour Thresholding

To determine the range of hues that needs to be thresholded, 2D histograms will be used.

The images will first be converted to HSV format, then a histogram will be generated based on the hue and saturation channels.

In [ ]:
for file in os.listdir("test_images/"):
    image = mpimg.imread(f"test_images/{file}")
    hsv = convert_to_hsv(image)
    # plt.imshow(hsv)  # shows that the image is indeed converted as imshow renders HSV channels as RGB channels
    histogram = calculate_hue_saturation_histogram(hsv)

    figure, (axis1, axis2) = plt.subplots(1, 2)
    axis1.imshow(histogram, interpolation = 'nearest')
    axis1.set_xlabel("Saturation")
    axis1.set_ylabel("Hue")

    axis2.imshow(image)

    figure.set_size_inches(20, 20)
    plt.show()

# Note: 0 <= hue <= 180, 0 <= saturation <= 255

From the histograms produced by the code above, we can see that the whites are usually at around 50 saturation or less, however the exact distribution seems to vary by image.

The yellows are at around a hue of 100. They are difficult to see, but there is a slightly lighter area at that location at around a saturation of 60 - 70. However, the yellow at such low saturation come from the hills, and it isn't apparent what the bright yellow's saturation is.

The large patch of datapoints at a hue of 20 and saturation of 90 - 140 come from the sky as no other colour has as much presence in the photos.

### The Pipeline

In [ ]:
def colour_thresholding(img, output=False):
    # Thresholds in the order: H, S, V
    WHITE_LOW_THRESHOLD = (0, 0, 200) 
    WHITE_HIGH_THRESHOLD = (180, 40, 255)
    YELLOW_LOW_THRESHOLD = (90, 110, 0)
    YELLOW_HIGH_THRESHOLD = (110, 180, 255)

    white_mask = cv2.inRange(img, WHITE_LOW_THRESHOLD, WHITE_HIGH_THRESHOLD)
    yellow_mask = cv2.inRange(img, YELLOW_LOW_THRESHOLD, YELLOW_HIGH_THRESHOLD)

    combined_mask = cv2.bitwise_or(white_mask, yellow_mask)
    colour_thresholded_image = cv2.bitwise_and(img, img, mask=combined_mask)
    if output:
        show_image(convert_to_rgb(colour_thresholded_image))
    return colour_thresholded_image

def image_pipeline(image, output=False):
    """Yanqi-made image processing pipeline, hope it works :fingers-crossed:"""
    raw_image = image
    hsv_image = convert_to_hsv(raw_image)

    if output:
        show_image(raw_image)
        show_image(hsv_image)

    # Colour thresholding (yellow and white)
    colour_thresholded_image = colour_thresholding(hsv_image, output=output)

    # Conversion to greyscale
    greyscale_image = grayscale(colour_thresholded_image)
    if output:
        show_image(greyscale_image, cmap="gray")

    # Smoothing
    smoothed_image = gaussian_blur(greyscale_image, 5)
    if output:
        show_image(smoothed_image, cmap="gray")

    # Canny edge detection
    edge_detect_image = canny(smoothed_image, 100, 100)
    if output:
        show_image(edge_detect_image, cmap="gray")

    # Region of interest filtering
    TOP_Y_VALUE = 320
    BOTTOM_Y_VALUE = 600
    X1 = 50
    X2 = 450
    X3 = 520
    X4 = 1000
    vertices = np.array([[X1, BOTTOM_Y_VALUE], [X2, TOP_Y_VALUE], [X3, TOP_Y_VALUE], [X4, BOTTOM_Y_VALUE]])
    region_of_interest_masked_image = region_of_interest(edge_detect_image, [vertices])
    if output:
        show_image(region_of_interest_masked_image, cmap="gray")

    # Line segment detection
    # According to the docs, rho and theta are the resolution in pixels and radians
    line_detected_image, lines = hough_lines(region_of_interest_masked_image, 1, np.pi/180, threshold=8, min_line_len=0, max_line_gap=15)
    if output:
        show_image(line_detected_image)
        show_image(weighted_img(line_detected_image, raw_image))

    # Classification into left and right boundaries
    middle_x_value = (X4 + X1) / 2
    left_points, right_points = group_lines_in_left_right(line_detected_image, lines, middle_x_value)
    if output:
        for points in [left_points, right_points]:
            line_img = np.zeros((line_detected_image.shape[0], line_detected_image.shape[1], 3), dtype=np.uint8)
            for index in range(0, len(points) - 1, 2):
                point1 = points[index]
                point2 = points[index + 1]
                cv2.line(line_img, point1, point2, color=[255, 0, 0], thickness=2)
            show_image(line_img)

    # Line fitting
    # Use RANSAC on the left and right line segments for it to plot a contiguous line representing the lane markings
    def run_ransac(x, y, x1, x2):
        ransac = create_ransac()
        ransac.fit(x, y)
        return ((int(x1), int(ransac.predict([[x1,],])[0][0])), (int(x2), int(ransac.predict([[x2,],])[0][0])))

    # Left
    x, y = np.split(left_points, indices_or_sections=2, axis=1)
    left_line_point = run_ransac(x, y, X1, X2)

    # Right
    x, y = np.split(right_points, indices_or_sections=2, axis=1)
    right_line_point = run_ransac(x, y, X3, X4)
    
    line_img = np.zeros((line_detected_image.shape[0], line_detected_image.shape[1], 3), dtype=np.uint8)
    for point in [left_line_point, right_line_point]:
        cv2.line(line_img, point[0], point[1], color=[255, 255, 255], thickness=5)

    # Output the final image where lanes are drawn on the raw image
    lanes_img = weighted_img(line_img, raw_image)
    if output:
        show_image(lanes_img)
    return lanes_img


# for file in os.listdir("test_images/"):
    # final_result_img = image_pipeline(mpimg.imread(f"test_images/{file}"))
    # show_image(final_result_img)
    # mpimg.imsave(f"test_images/{file}_result", final_result_img)

final_result_img = image_pipeline(mpimg.imread("test_images/solidYellowCurve.jpg"), output=True)


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    return image_pipeline(image)

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))